In [ ]:
from pyspark.sql import SparkSession
import pyspark
import glob
print (pyspark.__version__)


# --- Secrets/configs ---
s3_cfg = {
    "endpoint": "http://s3-minio:9900",
    "access_key": "kbmB5u0xCQs97d0vJxjx",
    "secret_key": "Tjq8PW7h0GU7rhiMrNMRrvCfpOhfIgvV8dwEhyIJ",
    "warehouse": "s3a://dwh/warehouse/"
}

jdbc_cfg = {
    "uri": "jdbc:postgresql://postgres:5432/iceberg",
    "user": "jdbc_cat",
    "password": "muppgpt"
}
jars = ",".join(glob.glob("/opt/freeds/spark/jars/*.jar"))

# --- Spark session setup ---
spark = (
    SparkSession.builder
    .appName("IcebergJDBCTest")
    # .master("spark://spark-master:7077")

    # Extra jars (Iceberg + Hadoop + Postgres)
    .config("spark.driver.extraClassPath", "/opt/freeds/spark/jars/*")
    .config("spark.executor.extraClassPath", "/opt/freeds/spark/jars/*")
    .config("spark.jars", jars)
    # Iceberg extensions
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")

    # JDBC catalog
    .config("spark.sql.catalog.freeds_cat", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.freeds_cat.type", "jdbc")
    .config("spark.sql.catalog.freeds_cat.uri", jdbc_cfg["uri"])
    .config("spark.sql.catalog.freeds_cat.warehouse", s3_cfg["warehouse"])
    .config("spark.sql.catalog.freeds_cat.jdbc.user", jdbc_cfg["user"])
    .config("spark.sql.catalog.freeds_cat.jdbc.password", jdbc_cfg["password"])

    # S3 (MinIO) access
    .config("spark.hadoop.fs.s3a.endpoint", s3_cfg["endpoint"])
    .config("spark.hadoop.fs.s3a.access.key", s3_cfg["access_key"])
    .config("spark.hadoop.fs.s3a.secret.key", s3_cfg["secret_key"])
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .getOrCreate()
)
# Print all Spark configuration settings
# for item in spark.sparkContext.getConf().getAll():
#     print(item)

spark.sql("CREATE SCHEMA IF NOT EXISTS freeds_cat.jaffle_kafe")

# --- Test Spark session ---
print("Available catalogs:")
for cat in spark.catalog.listCatalogs():
     print(cat)

# Example: create a table using Iceberg
spark.sql("""
CREATE TABLE IF NOT EXISTS freeds_cat.iceberg_db.test_table (
    id INT,
    name STRING
) USING ICEBERG
""")


3.5.6
Available catalogs:
CatalogMetadata(name='freeds_cat', description=None)
CatalogMetadata(name='spark_catalog', description=None)


DataFrame[]